In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
#importing the necessary libraries and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# Add the directory containing the Python file to the `sys.path` list
sys.path.append("/IMP DOCUMENT/College material/SEM6/AIProject/Ozone-Prediction-AIProject/Support")

# Import the Python file
import LSTM_Model
import TsPreProcessing

In [2]:
data = pd.read_excel('AnandVihar_CombinedData.xlsx', sheet_name='Sheet1')
data.head()
data2 = pd.read_excel('AnandVihar_2021-23.xlsx', sheet_name='Sheet1')
data2.head()
# give a list of column not in data from data2
not_common_cols = data2.columns.difference(data.columns)
not_common_cols
data2.drop(not_common_cols, axis=1, inplace=True)
data2.head()
final_data = pd.concat([data, data2], axis=0)
final_data.head()

,From Date,To Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,Benzene,Toluene,RH,WD,SR,BP,AT,RF,TOT-RF
0,01-04-2015 00:00,02-04-2015 00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
1,02-04-2015 00:00,03-04-2015 00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
2,03-04-2015 00:00,04-04-2015 00:00,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
3,04-04-2015 00:00,05-04-2015 00:00,53.96,166.93,18.27,55.11,70.93,25.78,30.99,1.61,25.63,2.04,10.27,59.87,131.07,256.38,740,23.28,None,0.0
4,05-04-2015 00:00,06-04-2015 00:00,100.82,260.41,15.91,50.65,62.23,32.94,47.61,1.42,23.2,4.39,10.6,69.64,124.15,174.72,740,20.74,None,0.0


In [3]:
# Convert the date column to datetime
from datetime import datetime
df = final_data.copy()
# datae
df['From Date'] = df['From Date'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))
df['To Date'] = df['To Date'].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))
df['From Date'].min(), df['From Date'].max()
df = df.set_index('To Date')
df.drop(['From Date'], axis=1, inplace=True)
df.head()

,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,Benzene,Toluene,RH,WD,SR,BP,AT,RF,TOT-RF
To Date,,,,,,,,,,,,,,,,,,
2015-04-02,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
2015-04-03,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
2015-04-04,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.0
2015-04-05,53.96,166.93,18.27,55.11,70.93,25.78,30.99,1.61,25.63,2.04,10.27,59.87,131.07,256.38,740,23.28,None,0.0
2015-04-06,100.82,260.41,15.91,50.65,62.23,32.94,47.61,1.42,23.2,4.39,10.6,69.64,124.15,174.72,740,20.74,None,0.0


In [9]:
# df.drop(['Temp'], axis=1, inplace=True)
num_cols = df.columns
num_cols
df = df.replace('None', np.nan)
# iterate through the columns and convert to float
for col in num_cols:
    df[col] = df[col].astype(float)
# dropping the null values
# i want to group by month
aq_df = df.copy()
aq_df.drop(['RF'], axis=1, inplace=True)
# ss.head()
aq_df['month'] = df.index.month
aq_df['year'] = df.index.year
aq_df['day'] = df.index.day
aq_df.head()
aq_df.isna().sum() / len(aq_df) * 100
aq_df.dropna(inplace=True)
aq_df.isnull().sum()


PM2.5      0
PM10       0
NO         0
NO2        0
NOx        0
NH3        0
SO2        0
CO         0
Ozone      0
Benzene    0
Toluene    0
RH         0
WD         0
SR         0
BP         0
AT         0
TOT-RF     0
month      0
year       0
day        0
dtype: int64

In [10]:
# define the train and test data
x_train = aq_df[:'2022-01-01']
y_train = x_train['Ozone']
x_test = aq_df['2022-01-01':]
y_test = x_test['Ozone']
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1761, 20), (1761,), (369, 20), (369,))

In [11]:
# transforming and normalizing the data
train_norm = TsPreProcessing.Transform_Normalize(x_train['Ozone'])
test_norm = TsPreProcessing.Transform_Normalize(x_test['Ozone'])
print(train_norm.shape)
print(test_norm.shape)

[0.16439404]
[0.14748852]
[0.05558648]
[0.0806317]
[0.07868373]
Number of null values in norm =  1
[0.07165077]
[0.09187059]
[0.05962256]
[0.09342596]
[0.06449606]
Number of null values in norm =  1
(1760,)
(368,)


In [12]:
n_steps = 15 #number of steps to look back
X_split_train, y_split_train = TsPreProcessing.split_sequence(train_norm, n_steps)
for i in range(len(X_split_train)):
    print(X_split_train[i], y_split_train[i])
n_features = 1 #number of features (in this case it is just the number of columns in the dataframe)
X_split_train = X_split_train.reshape((X_split_train.shape[0], X_split_train.shape[1], n_features))

[0.16439404 0.14748852 0.05558648 0.0806317  0.07868373 0.09308474
 0.07096146 0.05565605 0.0780576  0.12348685 0.12703492 0.08918881
 0.10094615 0.11075553 0.08070127] 0.09718936969528315
[0.14748852 0.05558648 0.0806317  0.07868373 0.09308474 0.07096146
 0.05565605 0.0780576  0.12348685 0.12703492 0.08918881 0.10094615
 0.11075553 0.08070127 0.09718937] 0.1296090162793933
[0.05558648 0.0806317  0.07868373 0.09308474 0.07096146 0.05565605
 0.0780576  0.12348685 0.12703492 0.08918881 0.10094615 0.11075553
 0.08070127 0.09718937 0.12960902] 0.13823570335327673
[0.0806317  0.07868373 0.09308474 0.07096146 0.05565605 0.0780576
 0.12348685 0.12703492 0.08918881 0.10094615 0.11075553 0.08070127
 0.09718937 0.12960902 0.1382357 ] 0.16738555725615692
[0.07868373 0.09308474 0.07096146 0.05565605 0.0780576  0.12348685
 0.12703492 0.08918881 0.10094615 0.11075553 0.08070127 0.09718937
 0.12960902 0.1382357  0.16738556] 0.13148740781967438
[0.09308474 0.07096146 0.05565605 0.0780576  0.12348685 0

In [13]:
X_split_test, y_split_test = TsPreProcessing.split_sequence(test_norm, n_steps)
for i in range(5):
    print(X_split_test[i], y_split_test[i])
n_features = 1
X_split_test = X_split_test.reshape((X_split_test.shape[0], X_split_test.shape[1], n_features))


[0.07165077 0.09187059 0.05962256 0.09342596 0.06449606 0.09197428
 0.09933637 0.06957694 0.10638739 0.09187059 0.11893405 0.15989216
 0.0736209  0.05993364 0.08627126] 0.11893405226047282
[0.09187059 0.05962256 0.09342596 0.06449606 0.09197428 0.09933637
 0.06957694 0.10638739 0.09187059 0.11893405 0.15989216 0.0736209
 0.05993364 0.08627126 0.11893405] 0.045831605143094145
[0.05962256 0.09342596 0.06449606 0.09197428 0.09933637 0.06957694
 0.10638739 0.09187059 0.11893405 0.15989216 0.0736209  0.05993364
 0.08627126 0.11893405 0.04583161] 0.058793031936955634
[0.09342596 0.06449606 0.09197428 0.09933637 0.06957694 0.10638739
 0.09187059 0.11893405 0.15989216 0.0736209  0.05993364 0.08627126
 0.11893405 0.04583161 0.05879303] 0.07476150974699294
[0.06449606 0.09197428 0.09933637 0.06957694 0.10638739 0.09187059
 0.11893405 0.15989216 0.0736209  0.05993364 0.08627126 0.11893405
 0.04583161 0.05879303 0.07476151] 0.1594773952716715


In [16]:
print(X_split_train.shape)

(1745, 15, 1)


In [ ]:
import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model



C:\Users\LENOVO\AppData\Local\Temp\ipykernel_11884\4243033194.py:58: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(build_fn=build_lstm_model_bn, epochs=20, batch_size=32, verbose=0)


ValueError: 
All the 243 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
243 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\wrappers\scikit_learn.py", line 155, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
TypeError: build_lstm_model_bn() missing 1 required positional argument: 'input_shape'
